In [1]:
import pandas as pd

from pyomo.environ import *
from pyomo.gdp import *

Ορίζω την διαδικασία παραγωγής του κάθε σταδίου της ταπετσαρίας, που απαιτεί τον χρόνο παραγωγής της αλλά και τις προαπαιτήσεις του. 

In [2]:
TASKS = {
    ('Wall_1','Blue')   : {'dur': 45, 'exists': None},
    ('Wall_1','Yellow') : {'dur': 10, 'exists': ('Wall_1','Blue')},
    ('Wall_2','Blue')   : {'dur': 20, 'exists': ('Wall_2','Green')},
    ('Wall_2','Green')  : {'dur': 10, 'exists': None},
    ('Wall_2','Yellow') : {'dur': 34, 'exists': ('Wall_2','Blue')},
    ('Wall_3','Blue')   : {'dur': 12, 'exists': ('Wall_3','Yellow')},
    ('Wall_3','Green')  : {'dur': 17, 'exists': ('Wall_3','Blue')},
    ('Wall_3','Yellow') : {'dur': 28, 'exists': None},
}

In [3]:
def define_model(TASKS):
    model = ConcreteModel()

    model.TASKS = Set(initialize = TASKS.keys(), dimen=2)
    model.JOBS = Set(initialize = list(set([j for (j,m) in model.TASKS])))
    model.MACHINES = Set(initialize = list(set([m for (j,m) in model.TASKS])))
    model.TASKORDER = Set(initialize = model.TASKS * model.TASKS, dimen=4,
        filter = lambda _, j, m, k, n: (k,n) == TASKS[(j,m)]['exists'])

    model.DISJUNCTIONS = Set(initialize = model.JOBS * model.JOBS * model.MACHINES, dimen=3,
        filter = lambda model, j, k, m: j < k and (j,m) in model.TASKS and (k,m) in model.TASKS)

    model.dur = Param(model.TASKS, initialize=lambda _, j, m: TASKS[(j,m)]['dur'])
    ub = sum([model.dur[j, m] for (j,m) in model.TASKS])
    model.makespan = Var(bounds=(0, ub))
    model.start = Var(model.TASKS, bounds=(0, ub))
    model.objective = Objective(expr = model.makespan, sense = minimize)
    model.finish = Constraint(model.TASKS, rule=lambda model, j, m:
        model.start[j,m] + model.dur[j,m] <= model.makespan)

    model.preceding = Constraint(model.TASKORDER, rule=lambda model, j, m, k, n:
        model.start[k,n] + model.dur[k,n] <= model.start[j,m])

    model.disjunctions = Disjunction(model.DISJUNCTIONS, rule=lambda model,j,k,m:
        [model.start[j,m] + model.dur[j,m] <= model.start[k,m],
         model.start[k,m] + model.dur[k,m] <= model.start[j,m]])

    TransformationFactory('gdp.hull').apply_to(model)
    return model



Δημιουργώ μία λύση χρησιμοποιώντας το gurobi με τις εξής μεταβλητές:

* Wallpaper: Ο αριθμός της ταπετσαρίας η οποία παράγεται
* Machine: το μηχάνημα που χρησιμοποείται για το βήμα της παραγωγής
* Start: Η ώρα που ξεκινάει το βήμα της παραγωγής
* Duration: Ο χρόνος που απαιτέιται για να ολοκληρωθεί το βήμα της παραγωγής
* Finish: Η ώρα που ολοκληρώνεται το βήμα της παραγωγής



In [4]:
model = define_model(TASKS)
SolverFactory('gurobi').solve(model)

results = [{'Wallpaper': w,
            'Machine': m,
            'Start': model.start[w, m](),
            'Duration': model.dur[w,m],
            'Finish': model.start[(w, m)]() + model.dur[w,m]}
            for w,m in model.TASKS]

order = pd.DataFrame(results)

print('Total time needed: ', ceil(order.Finish.max()))
print(order.sort_values(by=['Start']))

Total time needed:  97
  Wallpaper Machine  Start  Duration  Finish
3    Wall_2   Green    0.0        10    10.0
7    Wall_3  Yellow    0.0        28    28.0
2    Wall_2    Blue   10.0        20    30.0
5    Wall_3    Blue   30.0        12    42.0
4    Wall_2  Yellow   30.0        34    64.0
0    Wall_1    Blue   42.0        45    87.0
6    Wall_3   Green   80.0        17    97.0
1    Wall_1  Yellow   87.0        10    97.0
